In [6]:
import re

# Token types
token_patterns = [
    ('LITERAL', r'\d+'),
    ('OPERATOR', r'[\+\-\*/]'),
    ('PUNCTUATION', r'[\(\)]')
]


def tokenize(expression):
    tokens = []
    position = 0

    while position < len(expression):
        match = None
        for token_type, pattern in token_patterns:
            regex = re.compile(pattern)
            match = regex.match(expression, position)
            if match:
                lexeme = match.group(0)
                tokens.append({'type': token_type, 'lexeme': lexeme})
                break

        if match:
            position = match.end()
        else:
            invalid_char = expression[position]
            raise ValueError(f'Invalid token at position {position}: {invalid_char}')

    return tokens


def parse(tokens):
    index = 0

    def next_token():
        nonlocal index
        if index < len(tokens):
            token = tokens[index]
            index += 1
            return token
        else:
            return None

    def expression():
        token = next_token()
        if token and token['type'] == 'OPERATOR' and token['lexeme'] == '+':
            term()
            expression()
        else:
            index -= 1

    def term():
        token = next_token()
        if token and token['type'] == 'OPERATOR' and token['lexeme'] == '*':
            factor()
            term()
        else:
            index -= 1

    def factor():
        token = next_token()
        if token and token['type'] == 'PUNCTUATION' and token['lexeme'] == '(':
            expression()
            token = next_token()
            if token and token['type'] == 'PUNCTUATION' and token['lexeme'] == ')':
                return
            else:
                raise ValueError('Syntax error: Missing closing parenthesis')
        elif token and token['type'] == 'LITERAL':
            return
        else:
            raise ValueError('Syntax error: Unexpected token')

    expression()

    if index != len(tokens):
        raise ValueError('Syntax error: Extra tokens after parsing')


# Example expression
expression = '3 * (4 + @)'

# Tokenize the expression
try:
    tokens = tokenize(expression)
    print('Tokens:', tokens)

    # Parse the tokens
    try:
        parse(tokens)
        print('Syntax analysis successful')
    except ValueError as e:
        print('Syntax analysis failed:', str(e))
except ValueError as e:
    print('Lexical analysis failed:', str(e))

Lexical analysis failed: Invalid token at position 1:  
